# NVSHMEM Device API

We're finally ready to tackle the functionality we ultimately want from NVSHMEM: device-side memory operations.

## Returning to peer-to-peer operations

Earlier we saw that peer-to-peer GPU connections over NVLink support directly writing to or reading from peer GPU memory. (Indeed, this is what is done under the hood with a host-side call like `nvshmem_int_sum_reduce()`.) We used that to perform the CUDA operation
```
    atomicAdd(hits, 1);
```
even when `hits` pointed to device memory that wasn't local to the GPU performing the atomic add. How do we do the analogous operation in device code using NVSHMEM? Specifically, we want to achieve something similar to what we did in [exercises/monte_carlo_nvshmem_cuda_peer.cpp](exercises/monte_carlo_nvshmem_cuda_peer.cpp).

Fortunately, [NVSHMEM directly exposes atomic operations](https://docs.nvidia.com/nvshmem/api/gen/api/amo.html#). The one we're looking for is called [nvshmem_int_atomic_add](https://docs.nvidia.com/nvshmem/api/gen/api/amo.html#nvshmem-atomic-add) (although, for this use case, `nvshmem_int_atomic_inc` would also work). It looks a lot like CUDA's `atomicAdd`, but takes as a third argument the ID of the PE to do the atomic add to. Let's look at [exercises/monte_carlo_nvshmem_device.cpp](exercises/monte_carlo_nvshmem_device.cpp).

In [1]:
NUM_DEVICES = !nvidia-smi -L | wc -l
NUM_DEVICES = int(NUM_DEVICES[0])

In [2]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o monte_carlo_nvshmem_device solutions/monte_carlo_nvshmem_device.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_nvshmem_device

src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
Estimated value of pi = 3.14072
Error = 0.000277734


## Device-side collectives